In [1]:
# stdlib
import os

# Testing works over 4 possibilities
# 1. (python/in-memory workers and using tox commands)
# 2. (python/in-memory workers and manually running notebooks)
# 3. (using k8s and using tox commands)
# 4. (using k8s and manually running notebooks)
# Uncomment the lines below if in the 4th possibility

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["CLUSTER_HTTP_PORT_HIGH"] = "9081"
# os.environ["CLUSTER_HTTP_PORT_LOW"] = "9083"

In [2]:
# stdlib
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
high_port = os.environ.get("CLUSTER_HTTP_PORT_HIGH", "9081")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "9083")
print(environment, high_port, low_port)

python 9081 9083


In [3]:
# syft absolute
import syft as sy
from syft import test_settings
from syft.util.test_helpers.worker_helpers import (
    build_and_launch_worker_pool_from_docker_str,
)
from syft.util.test_helpers.worker_helpers import (
    launch_worker_pool_from_docker_tag_and_registry,
)

# Launch server & login

In [4]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port=high_port,
)

Autoreload enabled
Starting bigquery-low server on 0.0.0.0:9083


INFO:     Will watch for changes in these directories: ['/Users/koen/workspace/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:9083 (Press CTRL+C to quit)
INFO:     Started reloader process [18738] using WatchFiles


Found `reset=True` in the launch configuration. Resetting the server...
Stopping process on port: 9083
Process 18196 terminated.
Stopping process on port: 9083
Process 18656 terminated.


INFO:     Starting server with settings: {'name': 'bigquery-low', 'server_type': <ServerType.DATASITE: 'datasite'>, 'server_side_type': <ServerSideType.LOW_SIDE: 'low'>, 'deployment_type': <DeploymentType.PYTHON: 'python'>, 'processes': 1, 'reset': False, 'dev_mode': True, 'enable_warnings': False, 'in_memory_workers': True, 'queue_port': None, 'create_producer': True, 'n_consumers': 1, 'association_request_auto_approval': False, 'background_tasks': False, 'db_config': None, 'db_url': None} and worker class: <class 'syft.server.datasite.Datasite'>
INFO:     Started server process [18744]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Waiting for server to start Done.


...

INFO:     Will watch for changes in these directories: ['/Users/koen/workspace/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:9081 (Press CTRL+C to quit)
INFO:     Started reloader process [18752] using WatchFiles


Found `reset=True` in the launch configuration. Resetting the server...
Stopping process on port: 9081
Process 18204 terminated.
Stopping process on port: 9081
Process 18657 terminated.


INFO:     Starting server with settings: {'name': 'bigquery-high', 'server_type': <ServerType.DATASITE: 'datasite'>, 'server_side_type': <ServerSideType.HIGH_SIDE: 'high'>, 'deployment_type': <DeploymentType.PYTHON: 'python'>, 'processes': 1, 'reset': False, 'dev_mode': True, 'enable_warnings': False, 'in_memory_workers': True, 'queue_port': None, 'create_producer': True, 'n_consumers': 1, 'association_request_auto_approval': False, 'background_tasks': False, 'db_config': None, 'db_url': None} and worker class: <class 'syft.server.datasite.Datasite'>
INFO:     Started server process [18761]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Waiting for server to start Done.


SyftInfo: You have launched a development server at http://0.0.0.0:9081. It is intended only for local use.

In [5]:
# start email server here

In [6]:
low_client = server_low.login(email="info@openmined.org", password="changethis")

Logged into <bigquery-low: Low side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [7]:
high_client = server_high.login(email="info@openmined.org", password="changethis")

Logged into <bigquery-high: High side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [8]:
assert len(high_client.worker_pools.get_all()) == 1
assert len(low_client.worker_pools.get_all()) == 1

# Setup High First

- If using an external registery, we want to get this from the test_settings.
- We build the docker image over the base docker image in Syft
- We give a tag called worker-bigquery to our custom pool image

In [9]:
external_registry = test_settings.get("external_registry", default="docker.io")

base_worker_image = high_client.images.get_all()[0]

worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery 

""".strip()

docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)

In [10]:
worker_pool_name = "bigquery-pool"
custom_pool_pod_annotations = {"bigquery-custom-pool": "Pod annotation for bigquery"}
custom_pool_pod_labels = {"bigquery-custom-pool": "Pod_label_for_bigquery"}

In [11]:
build_and_launch_worker_pool_from_docker_str(
    environment=environment,
    client=high_client,
    worker_pool_name=worker_pool_name,
    custom_pool_pod_annotations=custom_pool_pod_annotations,
    custom_pool_pod_labels=custom_pool_pod_labels,
    worker_dockerfile=worker_dockerfile,
    external_registry=external_registry,
    docker_tag=docker_tag,
    scale_to=2,
)

Dockerfile ID: 8cc040b5f61f44b78d5e6553da342c67 successfully submitted.
[ContainerSpawnStatus(worker_name='bigquery-pool-1', worker=syft.service.worker.worker_pool.SyftWorker, error=None)]


In [12]:
assert len(high_client.worker_pools.get_all()) == 2

In [13]:
high_client.settings.allow_guest_signup(enable=False)

SyftSuccess: Registration feature successfully disabled

# Setup Low

In [14]:
launch_result = launch_worker_pool_from_docker_tag_and_registry(
    environment=environment,
    client=low_client,
    worker_pool_name=worker_pool_name,
    custom_pool_pod_annotations=custom_pool_pod_annotations,
    custom_pool_pod_labels=custom_pool_pod_labels,
    docker_tag=docker_tag,
    external_registry=external_registry,
    scale_to=1,
)

In [15]:
assert len(low_client.worker_pools.get_all()) == 2

# Register a DS only on the low side

In [16]:
low_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

SyftSuccess: User 'John Doe' successfully registered! To see users, run `[your_client].users`

In [17]:
low_client.settings.allow_guest_signup(enable=False)

SyftSuccess: Registration feature successfully disabled

In [18]:
assert (
    len(low_client.api.services.user.get_all()) == 2
), "Only DS and Admin should be at low side"
assert (
    len(high_client.api.services.user.get_all()) == 1
), "Only Admin should be at high side"

# Close

In [19]:
if environment != "remote":
    server_high.land()
    server_low.land()

Stopping bigquery-high


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18761]
INFO:     Stopping reloader process [18752]


Stopping bigquery-low


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18744]
INFO:     Stopping reloader process [18738]


In [ ]:
# TODO close email client